<a href="https://colab.research.google.com/github/AndreBrusco/RTDETR/blob/main/rtdetr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estruturando Google Drive:


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install beautifulsoup4 requests

from bs4 import BeautifulSoup
import requests
import os

# URL da pasta 'rgb' no repositório GitHub
url = "https://github.com/pedrozamboni/individual_urban_tree_crown_detection/tree/main/rgb"

# Fazer o scraping da página
response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encontrar os links dos arquivos na página
    files = soup.find_all('a', href=True)

    # Filtrar arquivos com extensão .png
    png_files = [
        file['href'].split('/')[-1]
        for file in files
        if file['href'].endswith('.png')
    ]

    print(f"Arquivos encontrados: {png_files}")

    # Diretório de destino no Google Drive
    dest_dir = '/content/drive/MyDrive/Imagens_Tree_Crowns/'
    os.makedirs(dest_dir, exist_ok=True)

    # URL base para os arquivos da pasta 'rgb'
    base_url = "https://github.com/pedrozamboni/individual_urban_tree_crown_detection/raw/main/rgb/"

    # Baixar os arquivos
    for file_name in png_files:
        file_url = f"{base_url}{file_name}"
        dest_file = os.path.join(dest_dir, file_name)
        os.system(f"wget -O {dest_file} {file_url}")
        print(f"Baixado: {file_name} -> {dest_file}")
else:
    print(f"Erro ao acessar a URL: {response.status_code}")


Arquivos encontrados: []


# **Colocando Imagens no Drive**

In [ ]:
#/content/RT-DETR/rtdetr_pytorch/configs/dataset/coco_detection.ymlimport os
import shutil

# Defina o caminho do Google Drive onde os arquivos serão armazenados
drive_base_path = '/content/drive/MyDrive/Imagens_Tree_Crowns'

# Repositório Git a ser clonado
repo_url = 'https://github.com/pedrozamboni/individual_urban_tree_crown_detection.git'
repo_clone_path = '/content/individual_urban_tree_crown_detection'

# Clonar o repositório
if not os.path.exists(repo_clone_path):
    print(f"Clonando o repositório {repo_url}...")
    os.system(f'git clone {repo_url}')
else:
    print(f"Repositório já clonado em {repo_clone_path}.")

# Pastas que queremos copiar do repositório
folders_to_copy = ['bbox_txt', 'gt', 'img_list', 'rgb']

# Loop para copiar cada pasta para o Google Drive
for folder_name in folders_to_copy:
    src_folder = os.path.join(repo_clone_path, folder_name)
    dest_folder = os.path.join(drive_base_path, folder_name)

    if os.path.exists(src_folder):
        # Criar a pasta de destino no Google Drive, se não existir
        if not os.path.exists(dest_folder):
            os.makedirs(dest_folder)

        # Copiar o conteúdo da pasta para o destino
        if not os.listdir(src_folder):
            print(f"Aviso: A pasta '{folder_name}' está vazia. Ignorando...")
        else:
            shutil.copytree(src_folder, dest_folder, dirs_exist_ok=True)
            print(f"Pasta '{folder_name}' copiada para {dest_folder}.")
    else:
        print(f"Aviso: A pasta '{folder_name}' não foi encontrada no repositório.")

# Opcional: Remover o repositório clonado para liberar espaço no Colab
if os.path.exists(repo_clone_path):
    shutil.rmtree(repo_clone_path)
    print(f"Repositório clonado em {repo_clone_path} foi removido.")

print("Todas as pastas foram copiadas para o Google Drive com sucesso!")


Clonando o repositório https://github.com/pedrozamboni/individual_urban_tree_crown_detection.git...
Pasta 'bbox_txt' copiada para /content/drive/MyDrive/Imagens_Tree_Crowns/bbox_txt.
Pasta 'gt' copiada para /content/drive/MyDrive/Imagens_Tree_Crowns/gt.
Pasta 'img_list' copiada para /content/drive/MyDrive/Imagens_Tree_Crowns/img_list.
Pasta 'rgb' copiada para /content/drive/MyDrive/Imagens_Tree_Crowns/rgb.
Repositório clonado em /content/individual_urban_tree_crown_detection foi removido.
Todas as pastas foram copiadas para o Google Drive com sucesso!


# Movendo os Arquivos na Estrutura COCO para a base COCO Train:

**Criação das Bases Train/Teste/Val**


In [ ]:
import os
import shutil
import json
from PIL import Image
import random

# Caminhos das pastas no Google Drive
base_dir = '/content/drive/MyDrive/Imagens_Tree_Crowns'
images_dir = os.path.join(base_dir, 'rgb')  # Diretório das imagens
annotations_dir = os.path.join(base_dir, 'bbox_txt')  # Diretório das anotações
img_list_dir = os.path.join(base_dir, 'img_list')  # Diretório das listas de conjuntos (train, val, test)

# Diretórios de saída
output_dir = '/content/drive/MyDrive/dataset'  # Saída para os arquivos COCO
output_images_dir = os.path.join(output_dir, 'images')
output_annotations_dir = os.path.join(output_dir, 'annotations')

# Limpar os diretórios de saída se já existirem
if os.path.exists(output_images_dir):
    shutil.rmtree(output_images_dir)
if os.path.exists(output_annotations_dir):
    shutil.rmtree(output_annotations_dir)

os.makedirs(output_images_dir, exist_ok=True)
os.makedirs(output_annotations_dir, exist_ok=True)

# Garantir que não há duplicatas entre os conjuntos
def split_and_remove_duplicates(images, train_ratio=0.6, val_ratio=0.2):
    """
    Divide os conjuntos em treino, validação e teste, garantindo que não haja duplicatas.
    """
    random.shuffle(images)  # Embaralhar imagens
    total_images = len(images)

    # Dividir os conjuntos
    train_split = int(total_images * train_ratio)
    val_split = int(total_images * (train_ratio + val_ratio))

    train_images = images[:train_split]
    val_images = images[train_split:val_split]
    test_images = images[val_split:]

    # Garantir que não haja duplicatas
    assert not set(train_images) & set(val_images), "Duplicatas entre treino e validação!"
    assert not set(train_images) & set(test_images), "Duplicatas entre treino e teste!"
    assert not set(val_images) & set(test_images), "Duplicatas entre validação e teste!"

    return train_images, val_images, test_images

# Processar conjuntos de dados
def process_set(set_name, image_files, used_images, verbose=True):
    """
    Processa um conjunto (train, val ou test), garantindo que não há duplicatas.
    """
    # Diretório para armazenar as imagens do conjunto
    set_images_dir = os.path.join(output_images_dir, set_name)
    os.makedirs(set_images_dir, exist_ok=True)

    images = []
    annotations = []
    annotation_id = 1  # ID único para cada anotação

    for image_id, image_file in enumerate(image_files, 1):
        if image_file in used_images:
            print(f"Aviso: Imagem duplicada encontrada '{image_file}', será ignorada.")
            continue

        used_images.add(image_file)

        # Caminhos das imagens
        src_image_path = os.path.join(images_dir, image_file)
        dst_image_path = os.path.join(set_images_dir, image_file)

        if not os.path.exists(src_image_path):
            print(f"Aviso: Imagem '{src_image_path}' não encontrada, ignorando...")
            continue

        # Copiar a imagem para o diretório correspondente
        shutil.copyfile(src_image_path, dst_image_path)

        # Obter dimensões da imagem
        with Image.open(src_image_path) as img:
            width, height = img.size

        # Adicionar entrada para a imagem
        images.append({
            'id': image_id,
            'file_name': image_file,
            'width': width,
            'height': height
        })

        # Ler o arquivo de anotações correspondente
        annotation_file = os.path.splitext(image_file)[0] + '.txt'
        annotation_path = os.path.join(annotations_dir, annotation_file)
        if not os.path.exists(annotation_path):
            print(f"Aviso: Arquivo de anotação '{annotation_path}' não encontrado para a imagem '{image_file}'.")
            continue

        # Processar cada linha do arquivo de anotações
        with open(annotation_path, 'r') as f:
            lines = f.readlines()

        for line in lines:
            # Separar os valores e garantir formato correto
            line = line.strip()
            if not line:
                continue
            parts = line.split()
            if len(parts) != 4:
                print(f"Aviso: Formato inválido na linha '{line}' do arquivo '{annotation_path}'.")
                continue

            # Converter coordenadas para floats
            try:
                x_min, y_min, x_max, y_max = map(float, parts)
            except ValueError:
                print(f"Aviso: Valores inválidos na linha '{line}' do arquivo '{annotation_path}'.")
                continue

            # Ajustar coordenadas para estarem dentro dos limites da imagem
            x_min = max(0, min(x_min, width - 1))
            y_min = max(0, min(y_min, height - 1))
            x_max = max(0, min(x_max, width - 1))
            y_max = max(0, min(y_max, height - 1))

            # Calcular largura e altura da bounding box
            bbox_width = x_max - x_min
            bbox_height = y_max - y_min

            if bbox_width <= 0 or bbox_height <= 0:
                print(f"Aviso: BBox com largura ou altura zero para a imagem '{image_file}'.")
                continue

            # Adicionar anotação
            annotations.append({
                'id': annotation_id,
                'image_id': image_id,
                'category_id': 0,  # ID da classe única
                'bbox': [x_min, y_min, bbox_width, bbox_height],
                'area': bbox_width * bbox_height,
                'iscrowd': 0
            })
            annotation_id += 1

    # Criar o JSON no formato COCO
    coco_format = {
        'images': images,
        'annotations': annotations,
        'categories': [
            {
                'id': 0,
                'name': 'Copa de Arvore',
                'supercategory': 'none'
            }
        ]
    }

    # Salvar o arquivo JSON
    json_file = os.path.join(output_annotations_dir, f'instances_{set_name}.json')
    with open(json_file, 'w') as f:
        json.dump(coco_format, f, indent=4)

    print(f"Conjunto '{set_name}' processado com sucesso! {len(images)} imagens e {len(annotations)} anotações salvas.")

# Listar todas as imagens disponíveis
all_images = os.listdir(images_dir)

# Garantir que não há duplicatas
train_images, val_images, test_images = split_and_remove_duplicates(all_images)

# Processar os conjuntos
used_images = set()  # Rastrear imagens já usadas
process_set('train', train_images, used_images)
process_set('val', val_images, used_images)
process_set('test', test_images, used_images)


Aviso: BBox com largura ou altura zero para a imagem '92.png'.
Aviso: BBox com largura ou altura zero para a imagem '40.png'.
Aviso: BBox com largura ou altura zero para a imagem '127.png'.
Aviso: BBox com largura ou altura zero para a imagem '208.png'.
Aviso: BBox com largura ou altura zero para a imagem '104.png'.
Aviso: BBox com largura ou altura zero para a imagem '197.png'.
Aviso: BBox com largura ou altura zero para a imagem '120.png'.
Aviso: BBox com largura ou altura zero para a imagem '122.png'.
Aviso: BBox com largura ou altura zero para a imagem '157.png'.
Aviso: BBox com largura ou altura zero para a imagem '154.png'.
Aviso: BBox com largura ou altura zero para a imagem '111.png'.
Aviso: BBox com largura ou altura zero para a imagem '166.png'.
Aviso: BBox com largura ou altura zero para a imagem '124.png'.
Aviso: BBox com largura ou altura zero para a imagem '151.png'.
Aviso: BBox com largura ou altura zero para a imagem '100.png'.
Aviso: BBox com largura ou altura zero par

In [ ]:
import os

train_images = set(os.listdir('./dataset/images/test/'))
val_images = set(os.listdir('./dataset/images/train/'))
val_images = set(os.listdir('./dataset/images/val/'))


duplicates = train_images & val_images
print(f"Imagens duplicadas entre treino e validação: {duplicates}")

Imagens duplicadas entre treino e validação: {'191.png', '156.png', '91.png', '127.png', '79.png', '5.png', '160.png', '194.png', '176.png', '162.png', '181.png', '56.png', '164.png', '175.png', '119.png', '186.png', '202.png', '159.png', '42.png', '132.png', '94.png', '157.png', '92.png', '8.png', '106.png', '217.png', '45.png', '173.png', '166.png', '21.png', '138.png', '196.png', '169.png', '62.png', '113.png', '201.png', '77.png', '22.png', '43.png', '190.png', '152.png', '198.png', '139.png', '109.png', '178.png', '40.png', '50.png', '219.png', '84.png', '31.png', '144.png', '146.png', '66.png', '206.png', '183.png', '65.png', '161.png', '81.png', '218.png', '51.png', '143.png', '1.png', '32.png', '165.png', '215.png', '174.png', '10.png', '208.png', '149.png', '170.png', '188.png', '203.png', '116.png', '17.png', '199.png', '212.png', '41.png', '145.png', '172.png', '35.png', '214.png', '134.png', '73.png', '83.png', '137.png', '48.png', '168.png', '185.png', '216.png', '147.png'

In [ ]:
# Instalar dependências necessárias
!pip install torchvision==0.15.2 torch==2.0.1
!pip install torch==2.0.1
!pip install torchvision==0.15.2
!pip install onnx==1.14.0
!pip install onnxruntime==1.15.1
!pip install pycocotools
!pip install PyYAML
!pip install scipy
import torchvision
print(f"Versão do Torchvision: {torchvision.__version__}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 117.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Selecione o Modelo a ser Executado

In [ ]:
!python /content/drive/MyDrive/RT-DETR/rtdetrv2_pytorch/tools/train.py \
    -c /content/drive/MyDrive/RT-DETR/rtdetrv2_pytorch/configs/rtdetrv2/rtdetrv2_r50vd_m_dsp_3x_coco.yml

2024-12-03 18:20:00.754251: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-03 18:20:00.771072: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-03 18:20:00.791932: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-03 18:20:00.798254: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-03 18:20:00.813253: I tensorflow/core/platform/cpu_feature_guar